# TODO

- [] first
- [x] second

# Main

In [3]:
import scrape
import random
import codecs
import pandas as pd

from requests import get
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from os import path, mkdir


In [2]:
def nu_idx(idx: str) -> list:
    """
    Pull all chapters from 'idx' series
    available on novelupdates.com and
    store them in a list of key-values
    """

    # Validate if idx is an url to novelupdates.com
    if urlparse(idx).netloc != 'www.novelupdates.com':
        raise Exception('Wrong input, use novelupdates.com series url.')

    base_url = get(idx + '?pg=1#myTable', headers=scrape.URL_HEADERS)

    chapters = []
    soup = BeautifulSoup(base_url, 'html.parser')

    while soup.find(class_='digg_pagination').find(class_='next_page') is not None:
        # What happens while there is a next page available
        for tag in soup.find(id='myTable').tbody.find_all('tr'):
            chapters.append({
            'date': tag.td.text.strip('\n').strip('\t'),
            'group': tag.find_all('a')[0].text,
            'groupu': tag.find_all('a')[0].attrs['href'],
            'chap': tag.find_all('a')[1].text, 
            'chapu': 'https:' + tag.find_all('a')[1].attrs['href']
            })

        # Wait between requests to avoid server overload and banhammer
        scrape.wait()
        next_url = get(idx + soup.find(class_='digg_pagination').find(class_='next_page').attrs['href'].strip('./'))
        # Update 'soup' variable to check all pages and break loop
        soup = BeautifulSoup(next_url, 'html.parser')

    # What happens when/if no next page is available
    for tag in soup.find(id='myTable').tbody.find_all('tr'):
        chapters.append({
        'date': tag.td.text.strip('\n').strip('\t'),
        'group': tag.find_all('a')[0].text,
        'groupu': tag.find_all('a')[0].attrs['href'],
        'chap': tag.find_all('a')[1].text, 
        'chapu': 'https:' + tag.find_all('a')[1].attrs['href']
        })

    # scrape.wait()
    chapters.reverse()
    return chapters


In [4]:
test_url = get('https://www.novelupdates.com/series/tensei-shitara-slime-datta-ken/?pg=29#myTable', headers=scrape.URL_HEADERS)
test = BeautifulSoup(test_url.content, 'html.parser')

chapters = []
for tag in test.find(id='myTable').tbody.find_all('tr'):
    chapters.append({
    'date': tag.td.text.strip('\n').strip('\t'),
    'group': tag.find_all('a')[0].text, # Group name
    'groupu': tag.find_all('a')[0].attrs['href'], # Group web url
    'chap': tag.find_all('a')[1].text, # Chapter name/number
    'chapu': 'https:' + tag.find_all('a')[1].attrs['href'] #Chapter url
    })

In [ ]:
for t in test.find(id='myTable').tbody.find_all('tr'):
    print(t.find_all('a')[1].attrs['title']) # Chapter name (alternate)

In [7]:
chapters[:2]

[{'date': '07/02/15',
  'group': 'Guro Translation',
  'groupu': 'https://www.novelupdates.com/group/guro-translation/',
  'chap': 'c21',
  'chapu': 'https://www.novelupdates.com/extnu/2533/'},
 {'date': '06/29/15',
  'group': 'Guro Translation',
  'groupu': 'https://www.novelupdates.com/group/guro-translation/',
  'chap': 'c20',
  'chapu': 'https://www.novelupdates.com/extnu/2131/'}]

## Fixing titles

In [10]:
with open('out.html', 'rb') as f:
    sopa = BeautifulSoup(f, 'html.parser')

# [list(a.attrs.values())[0] for a in sopa.find_all('a') if '/series/' in a.attrs['href']]
names = [list(a.attrs.values())[0] for a in sopa.find_all('a') if '/series/' in a['href']]
len(names)

31

In [3]:

def fix_names(name: str) -> str:
    new_name = name

    chars = {
    'ã\x80\x8e': codecs.decode(b'\xe3\x80\x8e'),
    'ã\x80\x8f': codecs.decode(b'\xe3\x80\x8f'),
    'Å\x8d': codecs.decode(b'\xc5\x8d'),
    'Ã\x98': codecs.decode(b'\xc3\x98'),
    }

    for wrong, right in chars.items():
        new_name = new_name.replace(wrong, right)
    
    with open('log.txt', 'a', encoding='utf-8') as log:
        if '\\' in r'%r' % new_name:
            print(f'Name not fixed: {r"%r" % name}')
            log.write(f'TODO: {r"%r" % name}\n')
        else:
            log.write(f'Cleaned: {name}\n')
            

    return new_name


In [4]:
import json

with open('readlist.json', 'r') as f:
    response = json.load(f)

table = [{'Name': t.attrs['title'], 'Url': t.attrs['href']} for t in BeautifulSoup(response['data'].replace('%','\\x')).tbody.find_all('a') if '/series/' in t.attrs['href']]
table[:5]


[{'Name': '(Um, Sorry) I’ve Been Reincarnated!',
  'Url': 'https://www.novelupdates.com/series/um-sorry-ive-been-reincarnated/'},
 {'Name': '100 Ways to Get the Male God',
  'Url': 'https://www.novelupdates.com/series/100-ways-to-get-the-male-god/'},
 {'Name': '101-banme no Hyaku Monogatari',
  'Url': 'https://www.novelupdates.com/series/101-banme-no-hyaku-monogatari/'},
 {'Name': '11 Ways to Forget Your Ex-Boyfriend',
  'Url': 'https://www.novelupdates.com/series/11-ways-to-forget-your-ex-boyfriend/'},
 {'Name': '11/23', 'Url': 'https://www.novelupdates.com/series/11-23/'}]

In [11]:
# 'https://www.novelupdates.com/series/an-alternate-worlds-strongest-canceller-/xef/xbd/x9e-using-the-rare-cancel-skill-to-become-godlike/xef/xbd/x9e/'
# print(codecs.decode(b'\xef'))
print(codecs.decode(b'\xef\xbc\x9e'))
# print(codecs.decode(b'\xc3\xaf\xc2\xbc\x9e'))

＞


In [136]:
bytes('\\'.encode('utf8'))

b'\\'

In [9]:
codecs.decode(b'\xe3\x80\x8e')

'『'

In [142]:
print(codecs.encode('ï¿¥1 Trillion Wife, Buy One Get One Free'))

b'\xc3\xaf\xc2\xbf\xc2\xa51 Trillion Wife, Buy One Get One Free'


In [141]:
acceptible = set(chr(i) for i in range(0x0021, 0xFFEF + 1))
# acceptible.issubset(set('ï¿¥1 Trillion Wife, Buy One Get One Free'))

In [ ]:
[r'%r' % name for name in [v['Name'] for v in table] if '\\' in r'%r' % name]

["'An Alternate World’s STRONGEST Canceller ~\\x9e Using The Rare Cancel Skill to Become GODLIKE~\\x9e'",
 "'Beastly FÄ\\x93i that Go Against the Heaven: Coerced by the HuÃ¡ng ShÅ«'",
 "'Cheat Aru Kedo Mattari Kurashitai ã\\x80\\x8aTensei Jinsei o Tanoshimou!ã\\x80\\x8b'",
 "'Dog God of the Fallen -SLASHDÃ\\x98G-'",
 "'Hiota no Kanojo ga Ore no Motteru Eroge ni Kyoumi Shinshin Nanda gaâ\\x80¦â\\x80¦'",
 "'Hon Issatsu de Kototariru Isekai RurÅ\\x8d Monogatari'",
 "'I Aim to Be an Adventurer with the Jobclass of â\\x80\\x9cJoblessâ\\x80\\x9d'",
 "'I Have Reincarnated intoã\\x80\\x8cWelcome to the â\\x97\\x8bâ\\x97\\x8b Town!ã\\x80\\x8dTownsperson A, But the Last Dungeon Was Right Around the Corner'",
 "'Is He A Hero? ã\\x80\\x8cYupã\\x80\\x8d'",
 "'Kami Sumeragi Yuusha no Eiyuutan ã\\x80\\x8aRyokou Tanã\\x80\\x8b'",
 "'Kawaranu Mono ï¼\\x9cKirameki no GÅ\\x8dremuï¼\\x9e'",
 "'Kujibiki Tokushou: Musou HÄ\\x81remu ken (WN)'",
 "'Kuzu Inouã\\x80\\x90Ondo wo Kaeru Monoã\\x80\\x91 no Ore ga M

## Random

In [ ]:
names_fix = [fix_names(name) if '\\' in r'%r' % name else name for name in [v['Name'] for v in table]]

In [36]:
names_fix[-10:]


['Yuusha Party no Kawaii Ko ga Ita no de, Kokuhaku Shite Mita (LN)',
 'Ze Tian Ji',
 'Zero no Tsukaima',
 'Zhan Long',
 'Zhu Xian',
 'Zhui Xu',
 'Zither Emperor',
 'Zombie Master',
 'ï¼\x9cInfinite Dendrogramï¼\x9e',
 'ï¿¥1 Trillion Wife, Buy One Get One Free']

In [95]:
print(names_fix[21])
print(names[21])

Shini Yasui KÅshaku ReijÅ to Shichi-nin no KikÅshi
Shini Yasui KÅshaku ReijÅ to Shichi-nin no KikÅshi


In [30]:
test = [random.randint(0,10) for n in range(10000)]
[f"Number {n} found {test.count(n)} times" for n in range(11)]

['Number 0 found 931 times',
 'Number 1 found 921 times',
 'Number 2 found 930 times',
 'Number 3 found 908 times',
 'Number 4 found 945 times',
 'Number 5 found 912 times',
 'Number 6 found 877 times',
 'Number 7 found 839 times',
 'Number 8 found 925 times',
 'Number 9 found 858 times',
 'Number 10 found 954 times']

# Placeholders

## Requests

In [4]:
agents = {'good': [], 'bad': []}

def fetch_user(bad = ''):
    import random

    if len(agents['good']) == 0:
        with open('user-agent.txt', 'r', encoding='utf8') as f:
            agents['good'] = [l for l in f]

    random.seed()
    random.choice(agents['good'])

'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 1.1.4322; InfoPath.2; .NET4.0E; .NET4.0C)\n'

In [21]:


# url = 'https://www.novelupdates.com/extnu/6961550/'

url_headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0) LinkCheck by Siteimprove.com'
}

# g = requests.get(url, headers=url_headers, allow_redirects=True)
# g = requests.get(url, headers=url_headers)

In [38]:
# g.history

[<Response [302]>]

## SQL

In [3]:
import sqlite3

In [5]:
novel_list = []

con = sqlite3.connect('novels.db')

cur = con.cursor()

for row in cur.execute('SELECT * FROM novels'):
    print(row)
    novel_list.append(row)

# con.commit()

('Death March kara Hajimaru Isekai Kyousoukyoku', 'https://www.sousetsuka.com/p/blog-page_15.html', 'Ongoing', None, None)


In [8]:
type(novel_list[0])

tuple

## DataFrame

In [5]:
import pandas as pd


desumachi_db = pd.DataFrame()

desumachi_db